In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import backtrader as bt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pyfolio as pf
import quantstats
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
import sys
from scipy.stats import rankdata
from scipy.stats import stats
from scipy.optimize import minimize

C:\ProgramData\Anaconda3\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


종목의 포트폴리오에 대한 위험 기여도(Risk Contribution)는 개별 종목 비중과 그 종목의 한계 위험 기여도(Marginal Risk Contribution)의 곱으로 나타납니다. 한계 위험 기여도는 종목 비중 1단위 증가 시 포트폴리오 변동성 증가량을 의미합니다. 위험 균형(Risk Parity) 전략은 포트폴리오 내 종목들의 위험 기여도가 같도록 하는 것이 목적입니다. 위험 기여도 계산을 위해서는 과거 수익률 데이터와 수익률 데이터의 공분산 행렬이 필요합니다. 미국 주식, 선진국 주식, 신흥국 주식, 전세계 채권, 미국 중기 국채, 미국 장기 국채, 미국 회사채, 미국 물가채, 금, 원자재, 미국 부동산(리츠) 데이터를 사용합니다.

In [2]:
start = '2006-08-01'
end = '2021-02-15'

vtsmx = web.DataReader("VTSMX", 'yahoo', start, end)['Adj Close'].to_frame("vtsmx")
vtmgx = web.DataReader("VTMGX", 'yahoo', start, end)['Adj Close'].to_frame("vtmgx")
veiex = web.DataReader("VEIEX", 'yahoo', start, end)['Adj Close'].to_frame("veiex")
pgbix = web.DataReader("PGBIX", 'yahoo', start, end)['Adj Close'].to_frame("pgbix")
vfitx = web.DataReader("VFITX", 'yahoo', start, end)['Adj Close'].to_frame("vfitx")
vustx = web.DataReader("VUSTX", 'yahoo', start, end)['Adj Close'].to_frame("vustx")
lqd = web.DataReader("LQD", 'yahoo', start, end)['Adj Close'].to_frame("lqd")
tip = web.DataReader("TIP", 'yahoo', start, end)['Adj Close'].to_frame("tip")
iau = web.DataReader("IAU", 'yahoo', start, end)['Adj Close'].to_frame("iau")
gsg = web.DataReader("GSG", 'yahoo', start, end)['Adj Close'].to_frame("gsg")
vgsix = web.DataReader("VGSIX", 'yahoo', start, end)['Adj Close'].to_frame("vgsix")

In [3]:
price_df = pd.concat([vtsmx, vtmgx, veiex, pgbix, vfitx, vustx, lqd, tip, iau, gsg, vgsix], axis=1)
return_df = price_df.pct_change().dropna(axis=0)

In [4]:
covmat = pd.DataFrame.cov(return_df)
covmat

,vtsmx,vtmgx,veiex,pgbix,vfitx,vustx,lqd,tip,iau,gsg,vgsix
vtsmx,0.000172,0.000158,0.000163,-0.000004,-0.000016,-0.000045,0.000010,-1.038789e-05,0.000005,8.837801e-05,0.000216
vtmgx,0.000158,0.000182,0.000176,-0.000004,-0.000014,-0.000041,0.000013,-6.561414e-06,0.000021,9.917475e-05,0.000193
veiex,0.000163,0.000176,0.000218,-0.000004,-0.000016,-0.000043,0.000013,-7.553274e-06,0.000022,1.107803e-04,0.000201
pgbix,-0.000004,-0.000004,-0.000004,0.000006,0.000004,0.000009,0.000005,4.401081e-06,0.000003,-3.153555e-06,-0.000005
vfitx,-0.000016,-0.000014,-0.000016,0.000004,0.000009,0.000021,0.000006,8.326408e-06,0.000006,-9.691218e-06,-0.000015
vustx,-0.000045,-0.000041,-0.000043,0.000009,0.000021,0.000068,0.000020,2.196225e-05,0.000016,-3.235389e-05,-0.000040
lqd,0.000010,0.000013,0.000013,0.000005,0.000006,0.000020,0.000031,9.942243e-06,0.000008,6.299333e-06,0.000013
tip,-0.000010,-0.000007,-0.000008,0.000004,0.000008,0.000022,0.000010,1.560001e-05,0.000011,9.473655e-07,-0.000011
iau,0.000005,0.000021,0.000022,0.000003,0.000006,0.000016,0.000008,1.122891e-05,0.000131,4.546024e-05,0.000006
gsg,0.000088,0.000099,0.000111,-0.000003,-0.000010,-0.000032,0.000006,9.473655e-07,0.000045,2.245168e-04,0.000093


비중과 공분산 행렬을 주면 위험 기여도(Risk Contribution)를 주는 함수입니다.

In [5]:
def RC(weight, covmat) :
    weight = np.array(weight)
    variance = weight.T @ covmat @ weight
    sigma = variance ** 0.5
    mrc = 1/sigma * (covmat @ weight)
    rc = weight * mrc
    rc = rc / rc.sum()
    return(rc)

가중치 x를 주면 위험 균형(Risk Parity)을 맞추기 위해서 각 자산별 위험 기여도 차이의 제곱합으로 쓰여지는 목적함수를 계산하고, 이론상으로 원하는 값인 0에 가까워지도록 최적화할 것입니다.

In [6]:
def RiskParityObjective(x) :
    
    variance = x.T @ covmat @ x
    sigma = variance ** 0.5
    mrc = 1/sigma * (covmat @ x)
    rc = x * mrc
    a = np.reshape(rc.to_numpy(), (len(rc), 1))
    risk_diffs = a - a.T
    # 1차원으로 풀어준 후에 제곱 -> 합 -> 목적함수
    sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
    return (sum_risk_diffs_squared)

레버리지가 없고, 공매도를 하지 않으며, 항상 보유 자금 전액 투자하는 것으로 가정하여 종목별 비중 합이 1이고, 각 종목 비중이 0 이상이라고 가정합니다.

In [7]:
# 제약 조건: 비중 합 1
def SumConstraint(x):
    return (x.sum()-1.0)

# 제약 조건: 비중 0 이상
def LongOnly(x):
    return(x)

실제 최적화하는 부분입니다. 초기 비중, 제약 조건, 반복 횟수 등을 설정합니다.

In [8]:
def RiskParity(covmat) :
    
    x0 = np.repeat(1/covmat.shape[1], covmat.shape[1]) 
    constraints = ({'type': 'eq', 'fun': SumConstraint},
                  {'type': 'ineq', 'fun': LongOnly})
    options = {'ftol': 1e-20, 'maxiter': 2000}
    
    result = minimize(fun = RiskParityObjective,
                      x0 = x0,
                      method = 'SLSQP',
                      constraints = constraints,
                      options = options)
    return(result.x)

위험 균형을 맞춘 종목별 가중치입니다. 대체로 변동성이 큰 주식, 리츠 자산군 비중이 낮고, 대체로 변동성이 낮은 국채 등 채권 자산군 비중이 높게 나옵니다.

In [9]:
RiskParity(covmat)

array([0.04310311, 0.03784878, 0.0352768 , 0.25211797, 0.2135053 ,
       0.08453533, 0.08353361, 0.12682572, 0.05369014, 0.04197046,
       0.02759279])

위험 균형이 실제 구현이 된 것인지 확인합니다. 위에서 만들어 둔 RC 함수로 각 자산별 위험 기여도를 체크합니다. 11개 자산 모두 위험 기여도가 같게 나옵니다.

In [10]:
weight_rp = RiskParity(covmat)

RC(weight_rp, covmat)

vtsmx    0.090909
vtmgx    0.090909
veiex    0.090909
pgbix    0.090909
vfitx    0.090909
vustx    0.090909
lqd      0.090909
tip      0.090909
iau      0.090909
gsg      0.090909
vgsix    0.090909
dtype: float64

비교를 위해 동일 가중으로 한 경우 어떻게 되는지 보겠습니다.

In [11]:
weight_equal = np.repeat(1/return_df.shape[1], return_df.shape[1])
rc_equal = RC(weight_equal, covmat)

rc_equal

vtsmx    0.165359
vtmgx    0.174111
veiex    0.187322
pgbix    0.002356
vfitx   -0.003196
vustx   -0.010265
lqd      0.030348
tip      0.008312
iau      0.061594
gsg      0.139916
vgsix    0.244142
dtype: float64

미국 주식(VTSMX), 선진국 주식(VTMGX), 신흥국 주식(VEIEX), 원자재(GSG), 미국 리츠(VGSIX) 5개 자산의 위험 기여도가 매우 높습니다. 비슷한 아이디어로, 자산배분의 기본 전략으로 알려진 주식 60 + 채권 40 전략은 주식이 대부분의 리스크를 가져갈 가능성이 높습니다. 실제 결과 상으로 미국 주식(VTSMX)이 99.975%, 미국 장기 국채(VUSTX)가 0.025% 리스크를 가져갑니다. 따라서, 채권 40%가 주식 60%의 위험을 제대로 막아주지 못하는 것입니다.

In [12]:
covmat_6040 = pd.DataFrame.cov(return_df[['vtsmx', 'vustx']])
weight_6040 = np.array([0.6, 0.4])
rc_6040 = RC(weight_6040, covmat_6040)

rc_6040

vtsmx    0.999751
vustx    0.000249
dtype: float64